In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Simple Newtwork


In [63]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
x= tf.placeholder(tf.float32,[None,784])

W_relu = tf.Variable(tf.truncated_normal([784,100],stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([100],stddev=0.1))
h = tf.nn.relu(tf.matmul(x,W_relu) + b_relu)

keep_probability = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h,keep_probability)

W = tf.Variable(tf.zeros([100,10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(h_drop,W) + b)

y_ = tf.placeholder(tf.float32, [None,10])

logit = tf.matmul(h_drop, W)+b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit,labels=y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    epoch = 2000
    for _ in range(epoch):
        batch_size = 100
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict = {x:batch_xs,y_:batch_ys, keep_probability:0.5})
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    a = sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels, keep_probability:1.})
    print(f'Accuracy {a}')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Accuracy 0.9642000198364258


Normalaze W

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [73]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
def fullyconnected_layer(tensor,input_size,out_size):
    W = tf.Variable(tf.truncated_normal([input_size,out_size],stddev=0.1))
    b = tf.Variable(tf.truncated_normal([out_size],stddev=0.1))
    return tf.nn.tanh(tf.matmul(tensor, W) + b)

def logit_layer(tensor, input_size, out_size):
    W = tf.Variable(tf.truncated_normal([input_size,out_size],stddev=0.1))
    b = tf.Variable(tf.truncated_normal([out_size],stddev=0.1))
    return tf.nn.softmax(tf.matmul(tensor, W) + b)

def batchnorm_layer(tensor, size):
    batch_mean, batch_var = tf.nn.moments(tensor,[0])
    beta = tf.Variable(tf.zeros([size]))
    scale = tf.Variable(tf.zeros([size]))
    return tf.nn.batch_normalization(tensor,batch_mean,batch_var,beta,scale,0.001)

x = tf.placeholder(tf.float32,[None,784])
h1 = fullyconnected_layer(x,784,100)
h1_bn = batchnorm_layer(h1,100)
h2 = fullyconnected_layer(h1_bn,100,100)
y_logit = logit_layer(h2,100,10)

y = tf.placeholder(tf.float32,[None,10])


loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = y_logit,labels=y))
train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

keep_probability = tf.placeholder(tf.float32)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_logit,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

init = tf.global_variables_initializer()

steps = 1000
batch_size = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x , batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op,feed_dict={x:batch_x,y:batch_y})
        if  i%100 == 0:
            a = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_probability:1.})
            print(f'Accuracy [{i}] = {a}')
    a = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
    print(f'Accuracy = {a}')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Accuracy [0] = 0.13369999825954437
Accuracy [100] = 0.9110999703407288
Accuracy [200] = 0.9229999780654907
Accuracy [300] = 0.9239000082015991
Accuracy [400] = 0.9312000274658203
Accuracy [500] = 0.9251999855041504
Accuracy [600] = 0.9272000193595886
Accuracy [700] = 0.9243999719619751
Accuracy [800] = 0.932200014591217
Accuracy [900] = 0.9226999878883362
Accuracy = 0.9210000038146973


Светочные сети

In [7]:
import numpy as np

In [24]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x,[-1,28,28,1])
W_conv_1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.1,shape=[32]))
conv_1 = tf.nn.conv2d(x_image,
                      W_conv_1,
                      strides = [1,1,1,1],
                      padding="SAME")+b_conv_1
h_conv_1 = tf.nn.relu(conv_1)
h_pool_1 = tf.nn.max_pool(h_conv_1,
                         ksize=[1,2,2,1],
                          strides=[1,2,2,1],
                          padding='SAME')
W_conv_2 = tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1,shape=[64]))
conv_2 = tf.nn.conv2d(h_pool_1,
                      W_conv_2,
                      strides=[1,1,1,1],
                      padding="SAME") + b_conv_2 
h_conv_2 = tf.nn.relu(conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2,
                          ksize=[1,2,2,1],
                          strides=[1,2,2,1],
                          padding='SAME')
h_pool_2_flat= tf.reshape(h_pool_2,[-1,7*7*64])

W_fc_1 =tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1,shape=[1024]))
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat,W_fc_1) + b_fc_1)

keep_probability = tf.placeholder(tf.float32)

h_fc_1_drop = tf.nn.dropout(h_fc_1,keep_probability)

W_fc_2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1,shape=[10]))
logit_conv = tf.matmul(h_fc_1_drop, W_fc_2)+b_fc_2
y_conv = tf.nn.softmax(logit_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit_conv,labels=y))

train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_conv,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

init = tf.global_variables_initializer()

steps = 5
batch_size = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x , batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_x,y:batch_y, keep_probability:0.5})
        if  i%1 == 0:
            a = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_probability:1.})
            print(f'Accuracy [{i}] = {a}')
    a = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_probability:1.})
    print(f'Accuracy = {a}')


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v

Keras Example

In [9]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
batch_size, img_rows,img_cols = 64,28,28
X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
X_test = X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
input_shape = (img_rows,img_cols,1)

In [16]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train/=255
X_test/=255

In [17]:
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)


In [23]:
model = Sequential()

model.add(Convolution2D(32,5,5,border_mode="same", input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),border_mode='same'))
model.add(Convolution2D(64,5,5,border_mode="same", input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),border_mode='same'))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
             )
model.fit(X_train,y_train,
          nb_epoch=5,
          verbose=1,
          validation_data=(X_test,y_test))
score= model.evaluate(X_test,y_test,verbose=0)
print(f'Test score: {score[0]}')
print(f'Test accuracy: {score[0]}')

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(28, 28, 1..., activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  after removing the cwd from sys.path.
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(28, 28, 1..., activation="relu", padding="same")`
  """
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  
/home/hvidsme

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 324s 5ms/step - loss: 0.0975 - accuracy: 0.9697 - val_loss: 0.0329 - val_accuracy: 0.9888
Epoch 2/5
60000/60000 [==============================] - 330s 6ms/step - loss: 0.0348 - accuracy: 0.9890 - val_loss: 0.0264 - val_accuracy: 0.9916
Epoch 3/5
60000/60000 [==============================] - 326s 5ms/step - loss: 0.0234 - accuracy: 0.9923 - val_loss: 0.0278 - val_accuracy: 0.9904
Epoch 4/5
60000/60000 [==============================] - 330s 5ms/step - loss: 0.0170 - accuracy: 0.9946 - val_loss: 0.0241 - val_accuracy: 0.9922
Epoch 5/5
60000/60000 [==============================] - 332s 6ms/step - loss: 0.0143 - accuracy: 0.9957 - val_loss: 0.0386 - val_accuracy: 0.9902
Test score: 0.038598290950577936
Test accuracy: 0.038598290950577936
